In [ ]:
# Analyse diff STAR HHV8 :
# install :

In [ ]:
import os
import pandas as pd

count_dir = "./count_star"
output_matrix = "./count_star/gene_count_matrix_STAR.csv"

files = [f for f in os.listdir(count_dir) if f.endswith("_ReadsPerGene.out.tab")]
dfs = []

for file in files:
    sample_name = file.replace("_not_hg38_ReadsPerGene.out.tab", "")  
    path = os.path.join(count_dir, file)


    df = pd.read_csv(path, sep='\t', skiprows=1, header=None)


    if len(df.columns) < 2:
        logging.warning(f"Fichier {file} a seulement {len(df.columns)} colonnes → skip")
        continue

    # Geneid + dernier colonne = nmb de reads
    df.columns = ['Geneid'] + [f"col{i}" for i in range(1, len(df.columns))]
    df = df[['Geneid', df.columns[-1]]]
    df.columns = ['Geneid', sample_name]

    dfs.append(df.set_index('Geneid'))

combined_df = pd.concat(dfs, axis=1)

# rm les  "unassigned"
gene_mask = ~combined_df.index.str.contains("N_", na=True) & ~combined_df.index.str.contains("unassigned", na=True)
combined_df = combined_df[gene_mask]


combined_df.to_csv(output_matrix)
print(f"Matrice Done  : {output_matrix}")